In [1]:
import os 
import re
from common.commonlib import file_subs, compile_VTM
import common.vvc_exec as vvc_exec
from pathlib import Path

class Simulation:
    cfg_dir = None
    vtm_dir = None
    out_dir = None

    version = 'Precise'
    qps = [22, 27, 32, 37]
    encoder = ['AI', 'RA', 'LD']

    n_frames = 32
    bg_exec = False
    gprof = False

    videos = []

    def __init__(self, n_frames = 32, version = 'Precise', qps = [22, 27, 32, 37], encoder = ['AI', 'RA', 'LD'], bg_exec = False, gprof = False):
        self.set_n_frames(n_frames)
        self.set_version(version)
        self.set_qps(qps)
        self.set_encoder(encoder)
        if gprof:
            self.enable_gprof() 
        else:
            self.disable_gprof()
        if bg_exec:
            self.enable_bg_exec()
        else:
            self.disable_bg_exec()

    def run_exec(self):
        print('Execution running')
        print(self.get_exec_info())
        for video in self.videos:
            for cfg in self.encoder:
                for qp in self.qps:
                    vvc_exec.exec(
                        vtm_dir         = self.vtm_dir, 
                        encoder_name    = cfg, 
                        cfg_video       = os.path.join(self.cfg_dir, video), 
                        qp              = qp,
                        out_dir         = self.out_dir,
                        video_name      = video[:-4], 
                        VTM_version     = self.version, 
                        n_frames        = str(self.n_frames),
                        gprof           = self.gprof,
                        background_exec = self.bg_exec
                    )
        print('Simulation done')

    def get_exec_info(self):
        info = \
            f'---------------------------------------------- \n' + \
            f'\n' + \
            f'version :         {self.version} \n' + \
            f'qps :             {self.qps} \n' + \
            f'encoder :         {self.encoder} \n' + \
            f'n_frames :        {self.n_frames} \n' + \
            f'background exec : {self.bg_exec} \n' + \
            f'gprof :           {self.gprof} \n' + \
            f'videos :          [ \n'
        for i, video in enumerate(self.videos):
            info += \
            f'                      {i:2}. {video}\n'
        info += f'                  ] \n' + \
                f'---------------------------------------------- \n'

        info += f'Total execution {len(self.videos)} x {len(self.qps)} x {len(self.encoder)} = {len(self.videos) * len(self.qps) * len(self.encoder)} simulations\n---------------------------------------------- \n'
        return info

    def remove_video_from_buffer(self, file_index):
        del self.videos[file_index]


    def replace_file(self, new_file, old_file):
        file_subs(new_file, old_file, Path(old_file).stem)
        compile_VTM(self.vtm_dir, os.getcwd())

    def set_n_frames(self, n_frames):
        self.n_frames = n_frames
    
    def set_out_dir(self, out_dir):
        self.out_dir = self.__create_output_dir__(out_dir)

    def set_vtm_dir(self, vtm_dir):
        self.vtm_dir = vtm_dir

    def set_cfg_dir(self, cfg_dir):
        self.cfg_dir = cfg_dir
        self.videos = self.__config_files_in_dir__(cfg_dir)

    def set_version(self, version):
        self.version = version

    def set_qps(self, qps):
        self.qps = qps
    
    def set_encoder(self, encoder):
        self.encoder = encoder

    def enable_gprof(self):
        self.gprof = True
    
    def disable_gprof(self):
        self.gprof = False

    def enable_bg_exec(self):
        self.bg_exec = True
    
    def disable_bg_exec(self):
        self.bg_exec = False

    def __create_output_dir__(self, output_dir):
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
        else:
            while(os.path.isdir(output_dir)):
                output_dir = self.__rename_dir__(output_dir)
            os.mkdir(output_dir)
        return output_dir

    def __config_files_in_dir__(self, cfg_vid_dir):
        if not os.path.isdir(cfg_vid_dir):
            raise Exception("Video config directory not exists")
        
        return [
            f 
            for f in os.listdir(cfg_vid_dir) 
            if os.path.isfile(os.path.join(cfg_vid_dir, f)) and 
            f[-4:] == '.cfg'
        ]
    def __rename_dir__(self, d : str ):
        matches = re.findall(r'(\d+)$', d)
        if len(matches) > 0:
            n = int(matches[-1])
            n = n + 1
            d = d[:-1] + str(n)
        else:
            d = d + '1'
        return d


In [2]:
s = Simulation(qps=[37], n_frames=8, encoder=['RA'])
s.set_cfg_dir('C:\\Users\\dudup\\OneDrive\\Área de Trabalho\\New folder\\VVC\\VVC_research\\database\\cfg-files\\')
s.set_vtm_dir('C:\\Users\\dudup\\OneDrive\\Área de Trabalho\\New folder\\VVC\\VVCSoftwareVTM\\')
s.set_out_dir('out')

for i in range(8,0,-1):
    s.remove_video_from_buffer(i)

print(s.get_exec_info())

---------------------------------------------- 

version :         Precise 
qps :             [37] 
encoder :         ['RA'] 
n_frames :        8 
background exec : False 
gprof :           False 
videos :          [ 
                       0. BasketballDrive.cfg
                  ] 
---------------------------------------------- 
Total execution 1 x 1 x 1 = 1 simulations
---------------------------------------------- 



In [6]:
s.run_exec()

Execution running
---------------------------------------------- 

version :         Precise 
qps :             [37] 
encoder :         ['RA'] 
n_frames :        8 
background exec : False 
gprof :           False 
videos :          [ 
                       0. BasketballDrive.cfg
                  ] 
---------------------------------------------- 
Total execution 1 x 1 x 1 = 1 simulations
---------------------------------------------- 


Encoding: ........... BasketballDrive

Video config: ....... C:\Users\dudup\OneDrive\Área de Trabalho\New folder\VVC\VVC_research\database\cfg-files\BasketballDrive.cfg
Encoder: ............ RA
QP: ................. 37
VTM version used: ... Precise
frames encoded: ..... 8

./"C:\Users\dudup\OneDrive\Área de Trabalho\New folder\VVC\VVCSoftwareVTM\bin\EncoderAppStatic" -c "C:\Users\dudup\OneDrive\Área de Trabalho\New folder\VVC\VVCSoftwareVTM\cfg\encoder_randomaccess_vtm.cfg" -c "C:\Users\dudup\OneDrive\Área de Trabalho\New folder\VVC\VVC_research\datab